<a href="https://colab.research.google.com/github/mahbubhimel/Transformer-NLP/blob/main/Multihead_Attention_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import tensorflow as tf

In [ ]:
sequence_length = 4
batch_size = 1
input_dim = 512
d_model = 512
x = tf.random.normal( (batch_size, sequence_length, input_dim) )

In [ ]:
x

In [ ]:
qkv_layer = tf.keras.layers.Dense(units=3 * d_model, activation=None, use_bias=True, input_shape=(input_dim,))

In [ ]:
qkv = qkv_layer(x)

In [ ]:
qkv

In [ ]:
num_heads = 8
head_dim = d_model // num_heads
qkv = tf.reshape(qkv, (batch_size, sequence_length, num_heads, 3 * head_dim))

In [ ]:
qkv

In [ ]:
qkv = tf.transpose(qkv, perm=[0, 2, 1, 3])

In [ ]:
qkv

In [ ]:
q, k, v = tf.split(qkv, num_or_size_splits=3, axis=3)

In [ ]:
q.shape, k.shape, v.shape

In [ ]:
k_transposed = tf.transpose(k, perm = [0, 1, 3, 2])

In [ ]:
d_k = q.shape[-1]

scaled = tf.matmul(q, k_transposed) / math.sqrt(d_k)
scaled.shape

In [ ]:
scaled

In [ ]:
# Create a mask with values set to negative infinity
mask = np.full(scaled.shape, -np.inf)

# Create an upper triangular mask by zeroing out values below the diagonal
mask = np.triu(mask, k=1)  # Keeps values above the diagonal

In [ ]:
mask

In [ ]:
scaled = scaled + mask

In [ ]:
scaled[0][0]

In [ ]:
np.exp(-0.5100664)/(np.exp(0.06729835)+np.exp(-0.5100664))

In [ ]:
attention = tf.nn.softmax(scaled, axis=-1)

In [ ]:
attention.shape

In [ ]:
attention[0][0]

In [ ]:
values = tf.matmul(attention, v)
values.shape

In [ ]:
values

**Attention Function**

In [ ]:
import numpy as np
import math
import tensorflow as tf

In [ ]:
def scaled_dot_product(q, k, v, mask=None):
  d_k = q.shape[-1]
  k_transposed = tf.transpose(k, perm = [0, 1, 3, 2])
  scaled = tf.matmul(q, k_transposed) / math.sqrt(d_k)

  #masking
  mask = np.full(scaled.shape, -np.inf)
  mask = np.triu(mask, k=1)

  if mask is not None:
    scaled = scaled + mask

  attention = tf.nn.softmax(scaled, axis=-1)
  values = tf.matmul(attention, v)

  return values, attention

In [ ]:
sequence_length = 4
batch_size = 1
input_dim = 512
d_model = 512
x = tf.random.normal( (batch_size, sequence_length, input_dim) )

qkv_layer = tf.keras.layers.Dense(units=3 * d_model, activation=None, use_bias=True, input_shape=(input_dim,))
qkv = qkv_layer(x)
num_heads = 8
head_dim = d_model // num_heads
qkv = tf.reshape(qkv, (batch_size, sequence_length, num_heads, 3 * head_dim))
qkv = tf.transpose(qkv, perm=[0, 2, 1, 3])
q, k, v = tf.split(qkv, num_or_size_splits=3, axis=3)

In [ ]:
values, attention = scaled_dot_product(q, k, v, mask= True)

In [ ]:
attention[0][0]

In [ ]:
values = tf.reshape(values, (batch_size, sequence_length, num_heads * head_dim))

In [ ]:
values.shape

In [ ]:
linear_layer = tf.keras.layers.Dense(units=d_model, activation=None, use_bias=True, input_shape=(d_model,))

In [ ]:
out = linear_layer(values)

In [ ]:
out.shape

# **Full Code Together**

In [ ]:
import numpy as np
import math
import tensorflow as tf

def scaled_dot_product(q, k, v, mask=None):
  d_k = q.shape[-1]
  k_transposed = tf.transpose(k, perm = [0, 1, 3, 2])
  scaled = tf.matmul(q, k_transposed) / math.sqrt(d_k)

  #masking
  mask = np.full(scaled.shape, -np.inf)
  mask = np.triu(mask, k=1)

  if mask is not None:
    scaled = scaled + mask

  attention = tf.nn.softmax(scaled, axis=-1)
  values = tf.matmul(attention, v)

  return values, attention

class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, input_dim, d_model, num_heads):
    super().__init__
    self.input_dim = input_dim
    self.d_model = d_model
    self.num_heads = num_heads
    self.head_dim = d_model // num_heads
    self.qkv_layer = tf.keras.layers.Dense(units=3 * d_model, activation=None, use_bias=True, input_shape=(input_dim,))
    self.linear_layer = tf.keras.layers.Dense(units=d_model, activation=None, use_bias=True, input_shape=(d_model,))

  def forward(self, x, mask = None):
    batch_size, sequence_length, input_dim = x.shape
    print(f"x.shape : {x.shape}")
    qkv = self.qkv_layer(x)
    print(f"qkv.shape : {qkv.shape}")
    qkv = tf.reshape(qkv, (batch_size, sequence_length, self.num_heads, 3 * self.head_dim))
    print(f"qkv.shape : {qkv.shape}")
    qkv = tf.transpose(qkv, perm=[0, 2, 1, 3])
    print(f"qkv.shape : {qkv.shape}")
    q, k, v = tf.split(qkv, num_or_size_splits=3, axis=3)
    print(f"q shape : {q.shape}, k shape : {k.shape}, v shape : {v.shape}")
    values, attention = scaled_dot_product(q, k, v, mask= None)
    print(f"values.shape : {values.shape}, attention.shape : {attention.shape}")
    values = tf.reshape(values, (batch_size, sequence_length, self.num_heads * self.head_dim))
    print(f"values.shape : {values.shape}")
    out = self.linear_layer(values)
    print(f"out.shape : {out.shape}")

    return out

In [ ]:
input_dim = 1024
d_model = 512
num_heads = 8

batch_size = 30
sequence_length = 5
x = tf.random.normal( (batch_size, sequence_length, input_dim) )

model = MultiHeadAttention(input_dim, d_model, num_heads)
out = model.forward(x)